# Explore reviewed detection history
Plot maps of number of individuals detected per point for each site and year

In [ ]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
def figsize(w,h):
    plt.rcParams['figure.figsize']=[w,h]
figsize(15,5) #for big visuals
%config InlineBackend.figure_format = 'retina'
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

In [ ]:
# Download the publicly available PAM dataset. Put the path to the folder here:
dataset_dir = "../../../pam_dataset_v4/"

In [ ]:
dethist = (
    pd.read_csv(
        f"{dataset_dir}/detection_history_complete.csv",
        index_col=0,
    )
    .drop(columns=["notes", "reviewer"])
    .reset_index(drop=True)
)
dethist.fillna(0, inplace=True)
dethist

In [ ]:
figsize(4, 1)
for year, yeardf in dethist.groupby("year"):
    counts, n = np.histogram(
        yeardf.drop(columns="year").set_index(["cluster_reviewed"]).sum(1),
        bins=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
    )
    n = n[:-1]
    plt.plot(
        n[1:],
        counts[1:],
        label=year,
        alpha=0.5,
        marker="o",
    )
    plt.xticks(
        n[1:],
    )
plt.title("number of days individual is detected in 12-day detection history")

plt.legend(loc="center left", bbox_to_anchor=(1, 0.5))

In [ ]:
all_cluster_samples = pd.read_csv(
    f"{dataset_dir}/all_clips_with_cleaned_clusters.csv",
    parse_dates=["datetime", "date", "time"],
)
all_cluster_samples["features3d"] = all_cluster_samples["features3d"].apply(eval)
all_cluster_samples["time"] = all_cluster_samples["time"].apply(lambda x: x.time())
all_cluster_samples["date"] = all_cluster_samples["date"].apply(lambda x: x.date())
all_cluster_samples["year"] = all_cluster_samples["date"].apply(lambda x: x.year)

In [ ]:
all_cluster_samples.groupby("year").size()

In [ ]:
all_cluster_samples["date1"] = all_cluster_samples["date"].apply(
    lambda x: x.replace(year=2000)
)
all_cluster_samples["year"] = all_cluster_samples.date.apply(lambda x: x.year)

### for a given point, visualize each individual's activity over the course of a season, for each year

In [ ]:
# visualize 10 points
n_individuals_per_point = all_cluster_samples.groupby(["point_code"])[
    "cluster_reviewed"
].nunique()
diverse_points = n_individuals_per_point[n_individuals_per_point > 3].index

import seaborn as sns

viz = all_cluster_samples[
    all_cluster_samples["point_code"].apply(lambda x: x in diverse_points)
]
# # Create the FacetGrid, ensuring data is divided by the 'year' column
g = sns.FacetGrid(data=viz, col="year", row="point_code", sharey=True, sharex=False)

# Map the `sns.histplot` to each subset of data
g.map_dataframe(sns.histplot, x="date", hue="cluster_reviewed", multiple="stack")

# Adjust the layout for clarity
g.set_axis_labels("Date", "Count")
g.set_titles("Year: {col_name}")
g.tight_layout()

# plt.savefig('../../figures/per_point_activity_across_dates.pdf')
plt.show()

In [ ]:
all_cluster_samples.groupby(["year", "site"])["cluster_reviewed"].nunique()

In [ ]:
all_cluster_samples.groupby(["year", "site"]).size()

In [ ]:
all_cluster_samples["date1"] = all_cluster_samples["date"].apply(
    lambda x: x.replace(year=2000)
)

In [ ]:
# load table of location and habitat variables
points = pd.read_csv(
    f"{dataset_dir}/habitat_measures_at_points.csv",
    index_col="point_code",
)

add coordinates to detection df:

In [ ]:
# all_cluster_samples["utm_E"] = all_cluster_samples.point_code.map(points["utm_E"])
# all_cluster_samples["utm_N"] = all_cluster_samples.point_code.map(points["utm_N"])
# all_cluster_samples["utm_zone"] = all_cluster_samples.point_code.map(
#     points["utm_zone"]
# ).astype(int)
# all_cluster_samples["utm_letter"] = all_cluster_samples.point_code.map(
#     points["utm_letter"]
# )
# all_cluster_samples["lon"] = all_cluster_samples.point_code.map(points["longitude"])
# all_cluster_samples["lat"] = all_cluster_samples.point_code.map(points["latitude"])

# map of N individuals per point for each site and year

In [ ]:
det_hist = (
    pd.read_csv(
        f"{dataset_dir}/detection_history_complete.csv",
        index_col=0,
    )
    .drop(columns=["notes", "reviewer"])
    .reset_index(drop=True)
    .set_index("cluster_reviewed")
    .fillna(0)
)
det_hist["point_code"] = det_hist.index.to_series().apply(lambda x: x.split("_")[0])
det_hist["lon"] = det_hist.point_code.map(points["longitude"])
det_hist["lat"] = det_hist.point_code.map(points["latitude"])
det_hist["site"] = det_hist.point_code.map(points["site"])
date_cols = det_hist.columns[1:-4]

In [ ]:
det_hist["detected"] = det_hist[date_cols].sum(axis=1) > 0
n_per_year = (
    det_hist[det_hist["detected"]]
    .reset_index()
    .groupby(["point_code", "year"])["cluster_reviewed"]
    .nunique()
)
n_per_year = n_per_year.unstack().fillna(0).astype(int)

In [ ]:
n_per_year["longitude"] = (
    n_per_year.reset_index()["point_code"].map(points["longitude"]).values
)
n_per_year["latitude"] = (
    n_per_year.reset_index()["point_code"].map(points["latitude"]).values
)
n_per_year["site"] = n_per_year.reset_index()["point_code"].map(points["site"]).values

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx

# Sample structure of n_per_year
# n_per_year = pd.read_csv("your_data.csv", index_col="point_code")

# Years (assume all except lon, lat, site are years)
# year_cols = [col for col in n_per_year.columns if col not in ["longitude", "latitude", "site"]]
sites = n_per_year["site"].unique()

# Convert to GeoDataFrame in Web Mercator
gdf_total = gpd.GeoDataFrame(
    n_per_year,
    geometry=gpd.points_from_xy(n_per_year.longitude, n_per_year.latitude),
    crs="EPSG:4326",
).to_crs(epsg=3857)

gdf_points = gpd.GeoDataFrame(
    points,
    geometry=gpd.points_from_xy(points.longitude, points.latitude),
    crs="EPSG:4326",
).to_crs(epsg=3857)
# gdf_singletons = gpd.GeoDataFrame(
#     n_singletons_per_year,
#     geometry=gpd.points_from_xy(n_singletons_per_year.lon, n_singletons_per_year.lat),
#     crs='EPSG:4326'
# ).to_crs(epsg=3857)

# Plot setup
year_cols = [2021, 2022, 2023, 2024]

n_rows = len(year_cols)
n_cols = len(sites)
fig, axes = plt.subplots(
    n_rows, n_cols, figsize=(5 * n_cols, 5 * n_rows), squeeze=False
)

for i, year in enumerate(year_cols):
    for j, site in enumerate(sites):
        ax = axes[i][j]
        # all points
        gdf_points[gdf_points.site == site].plot(
            ax=ax,
            markersize=1,
            alpha=1,
            facecolor="black",
            label="ARUs",
        )

        # Total counts
        total_subset = gdf_total[gdf_total["site"] == site].copy()
        total_subset["size"] = total_subset[year].fillna(0) * 20
        total_subset.plot(
            ax=ax,
            markersize=total_subset["size"],
            alpha=1,
            edgecolor="k",
            facecolor="blue",
            label="Total",
        )

        # Basemap and labels
        ctx.add_basemap(ax, source=ctx.providers.USGS.USTopo)
        ax.set_title(f"{site} - {year}")
        ax.set_axis_off()

        from matplotlib.lines import Line2D

        # Only add legend to first subplot
        if i == 0 and j == 0:
            legend_elements = [
                Line2D(
                    [0],
                    [0],
                    marker="o",
                    color="w",
                    label="individuals detected",
                    markerfacecolor="blue",
                    markersize=10,
                    markeredgecolor="k",
                ),
                Line2D(
                    [0],
                    [0],
                    marker="o",
                    color="black",
                    label="ARU without OVEN",
                    markerfacecolor="black",
                    markersize=1,
                    markeredgecolor="black",
                ),
            ]
            ax.legend(handles=legend_elements, loc="upper right")


legend_values = [1, 3, 5]  # These should correspond to your raw values
legend_sizes = np.array(legend_values) * 20  # Same scale as used above

legend_handles = [
    Line2D(
        [0],
        [0],
        marker="o",
        color="w",
        label=str(v),
        markerfacecolor="blue",
        markeredgecolor="k",
        markersize=np.sqrt(s),
    )  # Matplotlib uses sqrt(size)
    for v, s in zip(legend_values, legend_sizes)
]

# Add to plot
ax.legend(
    handles=legend_handles,
    title="individuals detected",
    loc="upper right",
    frameon=True,
)

plt.tight_layout()
plt.show()
# plt.legend()
plt.savefig("../../figures/FigureS9_maps_of_abundance_per_site_year.pdf")
plt.clf()